In [1]:
## IPython extensions to reload modules before executing user code.
# Autorelad is an IPython extension to reload modules before executing user code.
%load_ext autoreload

# Reload all modules (except those excluded by %aimport) every time before executing the Python code typed.
%autoreload 2

In [2]:
from matplotlib import pyplot as plt

import os
from os.path import join
import sys
from IPython.display import display
from PIL import Image

ROOT_DIR = os.sep.join(os.path.abspath('.').split(os.sep)[:-2])
sys.path.insert(0, ROOT_DIR)
DATASET_DIR = "{}/data".format(ROOT_DIR)
print(DATASET_DIR)

/home/djoy2409-wsl/projects/software_development/qsar_w_gnns/data


In [3]:
from typing import List, Union, Any, Tuple
from datetime import datetime
from random import sample

import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
import torch_geometric
from torch_geometric.data import Data
from torch.nn import BCELoss, LeakyReLU, ReLU
from torch.optim import lr_scheduler,Adagrad, Adadelta, Adam, AdamW, SGD

from rdkit import Chem, RDLogger
from rdkit.Chem import Draw, AllChem, PandasTools, MolFromSequence, MolToSmiles, MolToInchiKey, Descriptors, GraphDescriptors
from rdkit.Chem import rdMolDescriptors as rdmdesc

from joblib import Parallel, delayed

RDLogger.DisableLog('rdApp.*')
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
from lib import gnn_utils, utilities, datasets, splitters, featurizers, training_utils, graph_nns


from sklearn.metrics import f1_score, roc_auc_score, recall_score, precision_score, balanced_accuracy_score

import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 100
plt.rcParams.update({'font.size': 24})

In [4]:
## Set seeds
seed = 123
utilities.set_seeds(seed=seed) 

<p style="font-family: cursive; font-size=14px">
In the following, we will train regression models to predict the percentage of inhibition of the <em>P. falciparum's</em> DD2 growth.
</p>

<h3 style="font-family: cursive">Data Preparation</h4>

<h4 style="font-family: cursive">Dataset Cleaning</h4>
<p style="font-family: cursive; font-size=14px">
The dataset was obtained from the PubChem Biossay AID 2302. It was used as example in the paper by [Chakravarti and Mani Alla (2019); Frontiers in Artificial Intelligence, 2, DOI: 10.3389/frai.2019.00017](https://www.frontiersin.org/articles/10.3389/frai.2019.00017/full). </p>


In [5]:
# dd2_data_df   = pd.read_csv(f"{DATASET_DIR}/dd2_inhibition_pubchem_aid_2302.csv")
# smiles_column = "PUBCHEM_EXT_DATASOURCE_SMILES"
# # target_column = "PCT_INHIB_DD2"
# target_column = "DD2_ACTIVITY_CLS"
# mol_column    = 'RMol'

# print(f"Number of molecules with a missing SMILES string: {dd2_data_df[smiles_column].isna().sum()}/{dd2_data_df.shape[0]}")

# # Removing rows with a missing SMILES string
# dd2_data_df.dropna(subset=[smiles_column], axis=0, inplace=True)


# # Remove rows where is not defined
# dd2_data_df.dropna(subset=['PUBCHEM_ACTIVITY_OUTCOME'], axis=0, inplace=True)

# # Add a column with 0/1 activity values
# dd2_data_df[target_column] = dd2_data_df['PUBCHEM_ACTIVITY_OUTCOME'].apply(lambda x: 1 if x =='Active' else 0)


# # Adding molecule (Mol) objects to the dataframe
# PandasTools.AddMoleculeColumnToFrame(dd2_data_df, smiles_column, mol_column)

# dd2_data_df.head(2)

<span style="font-family: cursive; font-size=16px">
Beside removing rows with an empty SMILES, we will proceed with the following steps:
<ol>
    <li>Select the largest fragment for each molecule, which will also remove slats, among others. We will use the SMILES and inchikeys of the fragment.</li>
    <li>Sanitize the retained molecule</li>
    <li>Fix the target values. Negative values are set to 0 and values greater than 100 are set to 100</li>
    <li>Remove duplicates, using the inchikey as primary key. For each group of duplicates: </li>
        <ol>
            <li>If the min and max values are apart by 20 or more, omit and proceed to the next group</li>
            <li>Else, retain the first occurrence but replace the target value by an average</li>
            <li>It is worth mentioning that duplicate removal is usually a more involved process, where outliers can be detected and removed using a statisical approach, before averaging the taarget value.</li>
        </ol>
</ol>
</span>

In [6]:
# dd2_data_df['largest_frag']        = dd2_data_df[mol_column].apply(utilities.get_largest_fragment_from_mol)

# dd2_data_df['largest_frag_smiles'] = dd2_data_df['largest_frag'].apply(lambda mol: MolToSmiles(utilities.sanitize_molecule(mol, add_explicit_h=True)))

# dd2_data_df.dropna(subset=['largest_frag_smiles'], axis=0, inplace=True)


# dd2_data_df['largest_frag_ikey'] = dd2_data_df['largest_frag'].apply(MolToInchiKey)
# duplicates = dd2_data_df[dd2_data_df.duplicated(subset=['largest_frag_ikey'], keep=False)].sort_values(by='largest_frag_ikey')

# print(f"Number of molecules with a missing largest fragment SMILES string: {dd2_data_df['largest_frag_smiles'].isna().sum()}/{dd2_data_df.shape[0]}")

# print(f"Number of unique inchikeys: {dd2_data_df['largest_frag_ikey'].unique().size}/{dd2_data_df.shape[0]}")
# print(f"Number of duplicates inchikeys: {duplicates.shape[0]} - e.g.: {duplicates.index[:2]}")
# print(f"Number of molecules with a negative target value: {dd2_data_df[dd2_data_df[target_column]<0].shape[0]}/{dd2_data_df.shape[0]} - e.g.:{dd2_data_df[dd2_data_df[target_column]<0].index[:2]}")
# print(f"Number of molecules with target value>100: {dd2_data_df[dd2_data_df[target_column]>100].shape[0]}/{dd2_data_df.shape[0]} - e.g.:{dd2_data_df[dd2_data_df[target_column]>100].index[:2]}")
# dd2_data_df[['PUBCHEM_CID', 'largest_frag_ikey', target_column]].loc[[8,9,272, 458, 11041, 11042],:]



In [7]:
# dd2_data_df['lf_num_frags'] = dd2_data_df['largest_frag'].apply(lambda x: len(Chem.GetMolFrags(x)))
# print(dd2_data_df[dd2_data_df['lf_num_frags']>1])

In [8]:
# print(f" Unique target values = {dd2_data_df[target_column].unique()}")

# dd2_data_df['PUBCHEM_CID'] = dd2_data_df['PUBCHEM_CID'].astype(int)

# dd2_data_df[dd2_data_df.columns.difference([mol_column, 'largest_frag', 'lf_num_frags'])].to_csv(f"{DATASET_DIR}/dd2_inhibition_pubchem_aid_2302_cleaned.csv")
# dd2_data_df[['PUBCHEM_CID', 'largest_frag_smiles']].to_csv(f"{DATASET_DIR}/dd2_inhibition_pubchem_aid_2302_largest_frag_cleaned.csv")


# # dd2_data_df.head(2)

<p style="font-family: cursive; font-size=16px">As you can see, the target values outside of the [0,100] were fixed
    . Moreover, the rows with indices 11041 and 11042 are no longer in the datarame. This is because the corresponding rows have the same inchikeys, but with values that
    are more than 20 % apart. </p>

In [9]:
# atom_list = dd2_data_df['largest_frag'].apply(lambda mol: list(set([atom.GetAtomicNum() for atom in mol.GetAtoms()])))
# atom_list = list(set(utilities.flatten_list(atom_list.values.tolist())))

# print("List of unique atomic numbers: ",  [f"{i}:{Chem.Atom(i).GetSymbol()}" for i in atom_list])

atom_list = [35, 6, 7, 8, 9, 15, 16, 17, 53]

<p style="font-family: cursive; font-size=16px">Instead of defining a long range of values for possible atomic numbers (e.g.: 1-100), which could lead to a very sparse matrix, we can instead focus on the list of atoms that commonly occur in drug-like compounds. Specifically in this set, we notice that only 9 atoms occur in the list of tested compounds. If we add Hydrogen (which will be expicit), we have a vector of length 10 to represent the atomic number, plus 1 additional bit for unknown atomic numbers.</p>
<p style="font-family: cursive; font-size=16px">Le'ts define the atom anf bond featurizers.</p>


In [10]:
atom_featurizer = featurizers.AtomFeaturizer(
    allowable_sets_one_hot={
        "atomic_num": set(atom_list + [1]),
        "n_valence": {0, 1, 2, 3, 4, 5, 6},
        "n_hydrogens": {0, 1, 2, 3, 4},
        "hybridization": {"s", "sp", "sp2", "sp3", "sp3", "sp3d", "sp3d2"},
        "chiral_tag": {0,1,2,3},
        "is_aromatic": {True, False},
        "is_in_ring": {True, False} ,
        "is_in_ring_size_4": {True, False},
        "is_in_ring_size_5": {True, False},
        "is_in_ring_size_6": {True, False}
    }
    , continuous_props = ['atomic_mass', 'atomic_vdw_radius', 'atomic_covalent_radius']
)

bond_featurizer = featurizers.BondFeaturizer(
    allowable_sets_one_hot={
        "bond_type": {"single", "double", "triple", "aromatic"},
        "conjugated": {True, False},
        "stereo": {"stereonone, stereoz, stereoe, stereocis, stereotrans"}
    }
)

In [11]:
# print('atom_featurizer = ', atom_featurizer.features_mapping) 
# print('\nbond_featurizer = ', bond_featurizer.features_mapping) 

<h4 style="font-family: cursive"> Data splitting</h4>
<p style="font-family: cursive; font-size=16px">We will split the data into train, validation and test sets, using the scaffold splitting method. Small-molecule scaffold splitting, particularly using the Bemis-Murcko scaffold approach, involves identifying and extracting the core structural framework shared among a set of chemical compounds, excluding variable substituents. This process simplifies and abstracts the diverse chemical structures, aiding in the analysis of chemical libraries and structure-activity relationships. The Bemis-Murcko scaffold represents the most conserved structural elements of the compounds.<br/>While scaffold splitting facilitates the identification of key structural motifs,and can enhance model interpretation, it has limitations in handling highly flexible or structurally diverse molecules,and may result in the loss of detailed iformation about specific functional groups or substituents, potentially oversimplifying structure-activity relationships. Moreover, In some cases, scaffold splitting may lead to overly strict splitting criteria, resulting in training, validation, and test sets that are too similar in chemical space.</p>

In [12]:
# molecules = dd2_data_df['largest_frag'].values
# targets = dd2_data_df[target_column].values

# splitter= splitters.ScaffoldSplitter()
# train_idx, val_idx, test_idx = splitter.train_val_test_split(molecules=molecules[:], train_ratio=0.9, val_ratio=0.05
#                             , test_ratio=0.05, return_as_indices=True, return_as_clusters=False, include_chirality=False
#                             , sort_by_size=True, shuffle_idx=True, random_state=seed)

# # csplitter = splitters.ClusterSplitter()
# # train_idx, val_idx, test_idx = csplitter.train_val_test_split(molecules=molecules[:], train_ratio=0.8, val_ratio=0.1
# #                             , test_ratio=0.1, return_as_indices=True, return_as_clusters=False, include_chirality=True
# #                             , sort_by_size=False, shuffle_idx=False, random_state=seed, sim_cutoff=0.5)

# # kfold_splits = splitter.kfold_split(molecules=molecules[:50], n_folds=3, return_as_indices=False, include_chirality=False
# #                             , random_state=1)
# # print(f"fold sizes: {[len(f) for f in kfold_splits]}")

# train_mols, y_train = [molecules[i] for i in train_idx], targets[train_idx]
# val_mols, y_val     = [molecules[i] for i in val_idx], targets[val_idx]
# test_mols, y_test   = [molecules[i] for i in test_idx], targets[test_idx]

# print(f"Dataset sizes: Train ({len(train_mols)}) - Validation ({len(val_mols)}) - Test ({len(test_mols)})")

In [13]:
# Draw.MolsToGridImage(sample(train_mols, 5) +sample(val_mols, 5) + sample(test_mols, 5)
#                      , molsPerRow=5, subImgSize=(200, 150)
#                      , legends=['train'] * 5 + ['val'] * 5 + ['test'] * 5, highlightAtomLists=None)

In [14]:
%%time
# features_ = None
features_ = [
                'CalcExactMolWt'
                , 'CalcTPSA', 'CalcNumAromaticRings', 'CalcNumHBA', 'CalcNumHBD'
                , 'CalcNumRotatableBonds'
                # , 'CalcChi0n', 'CalcChi0v', 'CalcChi1n', 'CalcChi1v', 'CalcChi2n', 'CalcChi2v'
                # , 'CalcChi3n', 'CalcChi3v', 'CalcChi4n', 'CalcChi4v', 'CalcChi0n', 'CalcChi0v'
                , 'MolLogP', 'HallKierAlpha', 'qed', 'MaxPartialCharge', 'MinPartialCharge'
                # , 'ZagrebIndex'
                , 'MoeType'
            ]

mol_featurizer = featurizers.MoleculeFeaturizer(features = features_, df_func_gps=featurizers.DF_FUNC_GRPS_MINI) #featurizers.DF_FUNC_GRPS

# train_dataset  = gnn_utils.get_dataset_from_mol_list(mol_list=train_mols[:], targets=y_train[:]
#                                                     , atom_featurizer=atom_featurizer, bond_featurizer=bond_featurizer
#                                                     , mol_featurizer=mol_featurizer
#                                                     , add_explicit_h=True, compute_global_features=True
#                                                     , add_global_feat_to_nodes=False)

# print(train_dataset[1].x, train_dataset[1].y, train_dataset[1].x[0].shape) 
# if 'global_feats' in train_dataset[1].to_dict():
#     print(train_dataset[1].global_feats.shape)
# # train_dataset[1].x[5:10]

# # s = mol_featurizer.compute_rdkit_properties(train_mols[0])
# s

CPU times: user 75 µs, sys: 50 µs, total: 125 µs
Wall time: 123 µs


In [15]:
# val_dataset = gnn_utils.get_dataset_from_mol_list(mol_list=val_mols, targets=y_val
#                                                     , atom_featurizer=atom_featurizer, bond_featurizer=bond_featurizer
#                                                     , mol_featurizer=mol_featurizer
#                                                     , add_explicit_h=True, compute_global_features=True
#                                                     , add_global_feat_to_nodes=False)

# test_dataset = gnn_utils.get_dataset_from_mol_list(mol_list=test_mols, targets=y_test
#                                                     , atom_featurizer=atom_featurizer, bond_featurizer=bond_featurizer
#                                                     , mol_featurizer=mol_featurizer
#                                                     , add_explicit_h=True, compute_global_features=True
#                                                     , add_global_feat_to_nodes=False)

In [16]:
# dd2_dataset_fname = f'{DATASET_DIR}/dd2_datasets_objects_.pt'
# torch.save({
#             "splitter": splitter.__class__.__name__
#             , "train": train_dataset
#             , "val": val_dataset
#             , "test": test_dataset
#         }          
#         , dd2_dataset_fname
# )
# train_dataset[0]

In [17]:
dd2_dataset_fname = f'{DATASET_DIR}/dd2_datasets.pt'
dd2_dataset = torch.load(dd2_dataset_fname)  

train_dataset = dd2_dataset['train_dataset']
val_dataset   = dd2_dataset['val_dataset']
test_dataset  = dd2_dataset['test_dataset']

In [22]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

batch_size=128
train_loader, val_loader, test_loader =datasets.get_dataloaders(train_data=train_dataset, test_data=test_dataset
                                                                 , val_data=val_dataset, batch_size=batch_size, shuffle_train=True
                                                                 , add_global_feats_to_nodes=True, num_workers=0
                                                                 , scale_features=True, feature_scaler=StandardScaler())


dd2_loader_fname = f'{DATASET_DIR}/dd2_dataloaders_for_gin.pt'
torch.save({'train_loader':train_loader, 'val_loader':val_loader, 'test_loader':test_loader}, dd2_loader_fname)    
# list(train_loader)[0].x.shape

gfeatures_cleaned (10816, 104)
gfeatures_cleaned (676, 104)
gfeatures_cleaned (2028, 104)


In [23]:
# dd2_loader_fname = f'{DATASET_DIR}/dd2_dataloaders_for_gins.pt'
# dd2_loaders = torch.load(dd2_loader_fname)  

# train_loader = dd2_loaders['train_loader']
# val_loader   = dd2_loaders['val_loader']
# test_loader  = dd2_loaders['test_loader']

<h3 style="font-family: cursive">Implementing a Graph Isomorphism Network model</h3>
<p style="font-family: cursive">
A Graph Isomorphism Network (GIN) is a type of Graph Neural Network (GNN) that aggregates information from neighboring nodes and edges to learn node representations. GIN differs from other GNNs like Graph Attention Networks (GATs) and Graph Convolutional Networks (GCNs) by employing a readout function that integrates global graph-level information, allowing GIN to be more permutation invariant and capture graph isomorphism. GIN has been shown to be expressive and flexible, making it suitable for tasks where capturing structural similarities in graphs is crucial.</p>
<ol>
<li><a href="https://arxiv.org/abs/1810.00826">How Powerful are Graph Neural Networks?</li>
<li><a href="https://github.com/weihua916/powerful-gnns">How Powerful are Graph Neural Networks? (GitHub)</li>
<li><a href="https://medium.com/@mymomo119966.mm/graph-neural-networks-regressive-modelling-by-example-b13055bf56e8">Graph Isomorphism Neural Networks — Drug Discovery Example</li>
<li><a href="https://towardsdatascience.com/how-to-design-the-most-powerful-graph-neural-network-3d18b07a6e66">GIN: How to Design the Most Powerful Graph Neural Network</li>
</ol>

<h3 style="font-family: cursive">Creating and Training a model</h3>

In [24]:
%%time
import sklearn
from torch.optim import lr_scheduler

in_channels = list(train_loader)[0].x[0].shape[0]
global_fdim = list(train_loader)[0].global_feats.shape[1] if 'global_feats' in list(train_loader)[0].to_dict() else None
criterion   = BCELoss()

exp_params_grid = {
    'task': 'binary_classification','in_channels' : in_channels, 'global_fdim' : global_fdim,
    'model': ['graph_nns.GIN'],
    # 'gnn_hidden_neurons': [128, 256, 384, 512]
    'gnn_hidden_neurons': [200, 400]
    , 'gnn_nlayers':[4, 6]
    # , 'ffn_hidden_neurons': [64, 128, 256, 384, 512]
    , 'ffn_hidden_neurons': [100, 400]
    , 'ffn_nlayers': [1,2], 'out_neurons':1, 'dropout_rate': [0.1, 0.6]
    , 'activation_func': ['torch.nn.LeakyReLU', 'torch.nn.ReLU']
    , 'gpooling_func' : ['mean'],

    'learning_rate': [5e-2, 1e-2], 'n_epochs': [400, 700],
    'criterion' : criterion,
    'optimizer' : ["{'optimizer_type':'torch.optim.SGD', 'weight_decay':1e-3}"], 
    # 'scheduler' : ["{'lr_scheduler_type':'lr_scheduler.ReduceLROnPlateau', 'mode':'min', 'factor':0.7, 'patience':30, 'threshold':5e-02, 'eps':1e-08}"], 
    'scheduler' : ["{'lr_scheduler_type':'lr_scheduler.CyclicLR', 'max_lr':0.2, 'step_size_up':75, 'cycle_momentum':True}"], 

    'scoring_func': ['roc_auc_score'],
    'add_batch_norm':True
}

my_hpo = training_utils.OptunaHPO(n_trials=3, n_jobs=1, sampler=None)

results = my_hpo.run_optimization(   
            train_val_data= [train_loader, val_loader]
            , params_grid=exp_params_grid
            , optuna_direction='maximize'
            , split_mode='classic'
            , study_name='my_gin_study'
        )

results

[I 2024-10-22 17:28:03,138] A new study created in memory with name: my_gin_study


params = {'task': 'binary_classification', 'in_channels': 159, 'global_fdim': None, 'model': 'graph_nns.GIN', 'gnn_hidden_neurons': 328, 'gnn_nlayers': 6, 'ffn_hidden_neurons': 260, 'ffn_nlayers': 2, 'out_neurons': 1, 'dropout_rate': 0.5991433714786851, 'activation_func': 'torch.nn.ReLU', 'gpooling_func': 'mean', 'learning_rate': 0.04235075675058081, 'n_epochs': 400, 'criterion': BCELoss(), 'optimizer': "{'optimizer_type':'torch.optim.SGD', 'weight_decay':1e-3}", 'scheduler': "{'lr_scheduler_type':'lr_scheduler.CyclicLR', 'max_lr':0.2, 'step_size_up':75, 'cycle_momentum':True}", 'scoring_func': 'roc_auc_score', 'add_batch_norm': True}
in_channels = 159 :: gnn_dim = 328 :: activation_func = ReLU()
	optimizer:  {'state': {}, 'param_groups': [{'lr': 0.04235075675058081, 'momentum': 0, 'dampening': 0, 'weight_decay': 0.001, 'nesterov': False, 'maximize': False, 'foreach': None, 'differentiable': False, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,

[W 2024-10-22 17:28:44,590] Trial 0 failed with parameters: {'in_channels': 159, 'model': 'graph_nns.GIN', 'gnn_hidden_neurons': 328, 'gnn_nlayers': 6, 'ffn_hidden_neurons': 260, 'ffn_nlayers': 2, 'out_neurons': 1, 'dropout_rate': 0.5991433714786851, 'activation_func': 'torch.nn.ReLU', 'gpooling_func': 'mean', 'learning_rate': 0.04235075675058081, 'n_epochs': 400, 'optimizer': "{'optimizer_type':'torch.optim.SGD', 'weight_decay':1e-3}", 'scheduler': "{'lr_scheduler_type':'lr_scheduler.CyclicLR', 'max_lr':0.2, 'step_size_up':75, 'cycle_momentum':True}", 'scoring_func': 'roc_auc_score'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/djoy2409-wsl/anaconda3/envs/chemai-gpu/lib/python3.9/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/home/djoy2409-wsl/projects/software_development/qsar_w_gnns/lib/training_utils.py", line 679, in <lambda>
    objective = lambda trial: self.obj

KeyboardInterrupt: 

In [ ]:
# import json
# import inspect
# import ast
# d = json.loads('{"a":1, "b":2, "c":3}')
# del d["b"]
# print(d)
# # print(d = json.loads('{"a":1, "b":2, "c":3}'))
# d = '{"a":1, "b":5, "c":3}'
# print(ast.literal_eval(d))
# print("{'a':1, 'b':5}")

# prms = list(inspect.signature(torch.optim.SGD).parameters.keys())
# prms.remove('params')
# prms

# dir(torch.optim.lr_scheduler)

<h3 style="font-family: cursive">Training and Evaluating The Best Model</h3>

In [ ]:
best_params = results['best_params']
best_params.update(**{'task':'binary_classification'})
print("Best params: ", best_params)

gin_model = graph_nns.GIN.from_dict({
    "task" : best_params['task']
    , 'in_channels': best_params['in_channels'],  'gnn_hidden_neurons' : int(best_params['gnn_hidden_neurons'])
    , 'gnn_nlayers' : best_params['gnn_nlayers'], 'global_fdim' : best_params.get('global_fdim', None)
    , 'ffn_hidden_neurons' :int(best_params['ffn_hidden_neurons']), 'ffn_nlayers' : best_params['ffn_nlayers']
    , 'out_neurons' : best_params['out_neurons'], 'dropout_rate' : best_params['dropout_rate']
    , 'gpooling_func' : best_params['gpooling_func'], 'activation_func' : best_params['activation_func']
    , 'add_batch_norm' : True
})

<h4 style="font-family: cursive"> Model Training</h4>

In [ ]:
gnn_trainer = training_utils.GNNTrainer(task=best_params['task'], model=gcn_model
                                            , scheduler=best_params['scheduler']
                                            , optimizer=best_params['optimizer']
                                            , criterion=criterion
                                            , scoring_func=eval(best_params['scoring_func'])
                                            , n_epochs=best_params['n_epochs']
                                            , learning_rate=best_params['learning_rate']
                                        )
gnn_trainer.__dict__

In [ ]:
%%time
train_losses, val_losses, val_scores = gnn_trainer.train(train_loader=train_loader, val_loader=val_loader)

In [ ]:
utilities.plots_train_val_metrics(train_losses=train_losses, val_scores=val_scores
                            , val_losses=val_losses, figsize=(10, 7)
                            , image_pathname=f"{DATASET_DIR}/models/dd2_class_gnn_stats_gin.jpeg"
                            , val_score_name=best_params['scoring_func'])

NameError: name 'train_losses' is not defined

In [ ]:
torch.save(gin_model.state_dict(), f"{DATASET_DIR}/models/dd2_class_gin_model.pt")

<h4 style="font-family: cursive"> Model Evaluation</h4>

In [ ]:
from lib import predictions
test_pred, test_true = predictions.predict_from_loader(loader=test_loader, model=gin_model
                                                     , device='cuda:0'
                                                     , return_true_targets=True, desc="Predicting...")

threshold = 0.5
test_pred_classes = [int(x>threshold) for x in test_pred.squeeze(1)] 

# print(test_pred)
print(test_pred_classes)

print(f"\nROC AUC Score = {round(roc_auc_score(test_true.cpu(), test_pred.detach().cpu()),3)}")
print(f"Balanced Acc. = {round(balanced_accuracy_score(test_true.cpu(), test_pred_classes),3)}")
print(f"F1-Score      = {round(f1_score(test_true.cpu(), test_pred_classes),3)}")
print(f"Precision     = {round(precision_score(test_true.cpu(), test_pred_classes),3)}")
print(f"Recall        = {round(recall_score(test_true.cpu(), test_pred_classes),3)}")

<h5 style="font-family: cursive"> Evaluate using the GNNPredictor</h5>

In [ ]:
gin_predictor = predictions.GNNPredictor(model=gin_model, atom_featurizer=atom_featurizer
                                        , bond_featurizer=bond_featurizer, add_explicit_h=True
                                        , mol_featurizer=mol_featurizer
                                        , scale_features=True, feature_scaler=StandardScaler()
                                        , compute_global_features=True, add_global_feats_to_nodes=True)                                        

In [ ]:
# gin_model = torch.load(f"{DATASET_DIR}/models/dd2_class_gin_model.pt")
torch.save(gin_predictor, f"{DATASET_DIR}/models/dd2_class_gin_predictor.pt")

In [ ]:
gin_predictor=torch.load(f"{DATASET_DIR}/models/dd2_class_gin_predictor.pt")
gin_predictor

dd2_loader_fname = f'{DATASET_DIR}/dd2_dataloaders.pt'
dd2_loaders = torch.load(dd2_loader_fname)  

train_loader = dd2_loaders['train_loader']
val_loader   = dd2_loaders['val_loader']
test_loader  = dd2_loaders['test_loader']

In [ ]:
test_pred, test_true = gin_predictor.predict_from_loader(loader=test_loader
                                                     , device='cuda:0', return_true_targets=True, desc="Predicting...")


threshold = 0.5
test_pred_classes = [int(x>threshold) for x in test_pred.squeeze(1)] 
print(test_pred_classes)

print(f"\nROC AUC Score = {round(roc_auc_score(test_true.cpu(), test_pred.detach().cpu()),3)}")
print(f"Balanced Acc. = {round(balanced_accuracy_score(test_true.cpu(), test_pred_classes),3)}")
print(f"F1-Score      = {round(f1_score(test_true.cpu(), test_pred_classes),3)}")
print(f"Precision     = {round(precision_score(test_true.cpu(), test_pred_classes),3)}")
print(f"Recall        = {round(recall_score(test_true.cpu(), test_pred_classes),3)}")

<h5 style="font-family: cursive"> Predict DD2 Activity from SMILES</h5>

In [ ]:
smiles = ['CCC(=O)NCc1nccc1CO', 'O=C(O)c1cccnc1Sc1cc(CCO)c(Cl)cc1']
new_preds = gin_predictor.predict_from_smiles_list(smiles_list=smiles, device='cuda:0'
                                                , desc="Predicting...")

threshold = 0.5
pred_classes = [int(x>threshold) for x in new_preds.squeeze(1)]
print("\nnew_preds", new_preds)
print("pred_classes", pred_classes)